In [4]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from rastermap import Rastermap
from scipy.signal import savgol_filter
cmap = plt.cm.get_cmap('viridis_r')

import os
import scipy.io
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.stats import pearsonr, zscore
from scipy.io import savemat
import random
from utils import daily_blocks, extract_all_licks, get_tone_response_df, block_stim_evoke_response, load_behavior_txt
from sklearn.decomposition import PCA
plt.rcParams['axes.facecolor'] = 'white'
outcome_list = ['Hit', 'Miss', 'FA', 'CR']
mouse_names = ['jl009', 'fz017', 'fz009', 'fz006']
data_paths = ["Z:/Jenni/jl009/behavimaging.mat",
             "W:/Jenni/fz017/behavimaging.mat",
             "Y:/Jenni/fz009/behavimaging.mat",
             "Y:/sarah/fz006/behavimaging.mat"]
disks = ['Z', 'W', 'Y', 'Y']

mouse_id = 0
thresh = 0.05 # correlation threshold for selecting important axons
mouse_name = mouse_names[mouse_id]
disk = disks[mouse_id]

data_path = "{}_behavimaging.mat".format(mouse_name)
mat = scipy.io.loadmat(data_path)
data = mat['behavimaging']
nBlocks = data.shape[-1]
nDays = data[0, -1]['day'][0][0]
valid_days = []
for day in range(1, nDays+1):
    iblocks = daily_blocks(data, day)
    if len(iblocks)>0:
        valid_days.append(day)
        
# cmap = plt.cm.get_cmap('viridis_r')
if mouse_name == 'jl009':
    valid_days = [1, 2, 3, 5, 6, 7, 8, 9, 15]
    im_proportion = 0.3
if mouse_name == 'fz017':
    valid_days = [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 17, 23, 24, 25, 26]
    im_proportion = 0.2

In [5]:
# load lick data
# load resposne data

block = data[0, 0]
raw_spks = block['imagingdata']
NA, NT = raw_spks.shape
spks = zscore(raw_spks, axis=1)
# spks = sigmoid(spks)
nan_flag = np.isnan(np.sum(spks, axis=1))
not_nan_ind = np.where(nan_flag==False)[0]
spks = spks[not_nan_ind]

behavior = block['behavdata']
istim = np.where(behavior[0] > 0)[0]
cues = behavior[0, istim]
outcomes = behavior[1, istim]
ifirstlicks = -np.ones(len(istim))
licks = behavior[3]

print(licks.shape, spks.shape)

(13540,) (175, 13540)


# lick to response

In [6]:
# train the regressor for each point of response
# for each day
T = len(licks)
for t in range(T):
    x = licks[(t-30):(t+30)]
    y = spks[:, t] 

IndentationError: expected an indented block (7/ipykernel_37032/2074817477.py, line 3)

In [ ]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
# create datasets
# define model
model = LinearRegression()
# fit model
model.fit(X, y)
# make a prediction
row = [0.21947749, 0.32948997, 0.81560036, 0.440956, -0.0606303, -0.29257894, -0.2820059, -0.00290545, 0.96402263, 0.04992249]
yhat = model.predict([row])
# summarize prediction
print(yhat[0])

In [ ]:
# resp -> lick, sliding window